In [1]:
!pip install jupyter-dash

In [2]:
!pip install plotly==5.5.0

  Attempting uninstall: plotly
    Found existing installation: plotly 5.6.0
    Uninstalling plotly-5.6.0:
      Successfully uninstalled plotly-5.6.0


In [3]:
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from dash import no_update

In [47]:
URL="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [49]:
spacex_df=pd.read_csv(URL)

In [50]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [51]:
print(spacex_df.columns)

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')


In [52]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [53]:
from jupyter_dash import JupyterDash

In [54]:
app = JupyterDash(__name__)

In [55]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'}] + [
                                        {'label':l, 'value':l}
                                        for l in spacex_df['Launch Site'].unique().tolist()
                                    ],
                                    value="ALL",
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                 dcc.RangeSlider(id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [56]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data = spacex_df
        fig = px.pie(data, values='class', names='Launch Site',
                title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        data = (spacex_df
            .loc[spacex_df['Launch Site'] == entered_site]
            .groupby('class')[['class']]
            .count()
            .rename({"class":"class_count"}, axis=1)
            .reset_index()
        )
        fig = px.pie(data, values='class_count', names='class',
                title=f'Total Success Launches for site "{entered_site}"')
        return fig

In [57]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [
                  Input(component_id='site-dropdown', component_property='value'),
                  Input(component_id='payload-slider', component_property='value')
              ])
def get_scatter_plot(entered_site, entered_payload):
    min_value = entered_payload[0]
    max_value = entered_payload[1]
    data = spacex_df.loc[
        (spacex_df['Payload Mass (kg)'] >= min_value) &
        (spacex_df['Payload Mass (kg)'] <= max_value)
    ]
    if entered_site == 'ALL':
        fig = px.scatter(data, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                title=f'Correlation between Payload and Success for all Sites')
        return fig
    else:
        data = data.loc[data['Launch Site'] == entered_site]
        fig = px.scatter(data, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                title=f'Correlation between Payload and Success for site "{entered_site}"')
        return fig

In [58]:
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

